# TABLE PERSON

This table is composed by 3 tables:

- <code>train_person_1</code> with depth=1
- <code>train_person_2</code> with depth=2


We will analyze this points:

- the columns of all dataframes
- how to merge them
- their NA meanings and how to fill them
- some plots
- how to create some behavioural KPI.

# 1. SETTINGS

In [ ]:
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
import os 

sys.path.append("../../")
from src.utils import get_feature_definitions

In [ ]:
dataPath = "../../data/"

In [ ]:
df_feature_definition = pl.read_csv(dataPath + "feature_definitions.csv")        

In [ ]:
table_person_1 = pl.read_parquet(dataPath + "parquet_files/train/train_person_1.parquet")
table_person_2 = pl.read_parquet(dataPath + "parquet_files/train/train_person_2.parquet")

# 2. TRAIN PERSON 1 

In [ ]:
with pl.Config() as cfg:
    cfg.set_fmt_str_lengths(150)
    cfg.set_tbl_rows(-1)

    display(get_feature_definitions(table_person_1.columns, df_feature_definition)) 

# 3. TRAIN PERSON 2

In [ ]:
table_person_2_pd = table_person_2.to_pandas()

In [ ]:
with pl.Config() as cfg:
    cfg.set_fmt_str_lengths(150)
    cfg.set_tbl_rows(-1)

    display(get_feature_definitions(table_person_2.columns, df_feature_definition)) 

In [ ]:
number_of_persons = table_person_2.group_by("case_id").agg(
    pl.col("num_group2").max().alias("number_of_persons")
).sort("number_of_persons", descending=True)

summary_status = table_person_2.filter(pl.col("num_group1")==0).sort("num_group2", descenging=False).group_by("case_id").agg(
    pl.col("empls_employedfrom_796D").min().alias("start_to_work"),
    pl.col("empls_employer_name_740M").n_unique().alias("number_of_employers"),
    pl.col("empls_employedfrom_796D").n_unique().alias("number_of_economical_status_changes"),
    
    pl.col("addres_zip_823M").n_unique().alias("number_of_house_changes"),    
    (pl.col("addres_role_871L") == "PERMANENT").max().alias("is_permanent_work"),
    (pl.col("addres_role_871L") == "TEMPORARY").max().alias("is_temporary_work"),
    (pl.col("addres_role_871L") == "PERMANENT").last().alias("is_permanent_last_work"),
    (pl.col("addres_role_871L") == "TEMPORARY").last().alias("is_temporary_last_work"),
    

)

In [ ]:
table_person_2_agg_conts_role = table_person_2.filter(pl.col("num_group1") == 0).sort("num_group2", descending=False).group_by("case_id").agg(
        pl.col("conts_role_79M").last().alias("last_conts_role_79M"),
        pl.col("conts_role_79M").first().alias("first_conts_role_79M"),
        pl.col("conts_role_79M").mode().alias("most_frequent_conts_role_79M"),

        pl.col("empls_economicalst_849M").last().alias("last_empls_economicalst_849M"),
        pl.col("empls_economicalst_849M").first().alias("first_empls_economicalst_849M"),
        pl.col("empls_economicalst_849M").mode().alias("most_frequent_empls_economicalst_849M"),

        
        pl.col("empls_employer_name_740M").last().alias("last_empls_employer_name_740M"),
        pl.col("empls_employer_name_740M").first().alias("first_empls_employer_name_740M"),
        pl.col("empls_employer_name_740M").mode().alias("most_frequent_empls_employer_name_740M"),

        pl.col("addres_role_871L").last().alias("last_addres_role_871L"),
        pl.col("addres_role_871L").first().alias("first_addres_role_871L"),
        pl.col("addres_role_871L").mode().alias("most_frequent_addres_role_871L"),
        
        pl.col("addres_zip_823M").last().alias("last_addres_zip_823M"),
        pl.col("addres_zip_823M").first().alias("first_addres_zip_823M"),
        pl.col("addres_zip_823M").mode().alias("most_frequent_addres_zip_823M"),


        pl.col("addres_district_368M").last().alias("last_addres_addres_district_368M"),
        pl.col("addres_district_368M").first().alias("first_addres_addres_district_368M"),
        pl.col("addres_district_368M").mode().alias("most_frequent_addres_district_368M"),

)


## 3.1 TABLE PERSON 2:NULL ANALYSIS

In [ ]:
df_nulls = (table_person_2.null_count() / table_person_2.shape[0]).transpose(include_header=True).sort(by="column_0", descending=True).to_pandas()
df_nulls["perc_of_nulls"] = df_nulls.iloc[:, 1] 
df_nulls = df_nulls.drop("column_0", axis = 1)
df_nulls

The only null information are related with:
- start of employment
- related people
- address role


In [ ]:
case_id_without_start_empl = table_person_2.filter(pl.col("num_group1") == 0).filter(pl.col("empls_employedfrom_796D").is_null()).select("case_id").unique()


### 3.2 TABLE: PERSON2: COLUMN ANALYSIS

In [ ]:
features_num = list(table_person_2_pd.select_dtypes('number'))
features_total = table_person_2_pd.columns.tolist()
features_date = [el for el in features_total if el.endswith("D")]
features_cat = [el for el in features_total if el not in (features_num + features_date)]
features_num.remove('case_id')

In [ ]:
for col in features_num:
    print(col, ": ", len(table_person_2_pd[col].unique()))

In [ ]:
for col in features_cat:
    print(col, ": ", len(table_person_2_pd[col].unique()))

In [ ]:
for col in features_cat:
    print(col, ": ", len(table_person_2_pd[col].unique()))

In [ ]:
for col in features_date:
    table_person_2_pd[col] = pd.to_datetime(table_person_2_pd[col], errors='coerce')
    min_date, max_date = table_person_2_pd[col].min(), table_person_2_pd[col].max()
    print(f"Col: {col}, Min: {min_date}, Max: {max_date}")

# 4. FINAL AGGREGATION

In [ ]:
table_person_2.filter(table_person_2["case_id"] == 2562911	)

In [ ]:
table_person_2.groupby("case_id").agg(pl.col("addres_zip_823M").n_unique()).sort(by="addres_zip_823M", descending=True)


In [ ]:
table_person_2.groupby("case_id").agg(pl.col("addres_district_368M").n_unique()).sort(by="addres_district_368M", descending=True)


In [ ]:
table_person_2.groupby("case_id").agg(
    [
        pl.col("addres_district_368M").unique()
        
    ]
)

In [ ]:
sys.exit()

In [ ]:
table_person_2

! POSSIBLE AGGREGATION:
- "empls_employedfrom_796D" -> we take the min date. 
- "empls_employer_name_740M" -> do frequency encoding and associate with the related date (if possible)
- "empls_economicalst_849M" -> same as above. Take the week if possible and associate with the loan.
- "relatedpersons_role_762T" -> create how many related person the loan have 

In [ ]:
table_person_2.filter(table_person_2["case_id"] == 2702362)

In [ ]:
table_person_2.filter(~table_person_2["relatedpersons_role_762T"].is_null()).sort("case_id")

In [ ]:
table_person_2["relatedpersons_role_762T"].unique()